In [ ]:
#modelling of second model
#those that were sold elsewhere but not in these shops
#do you drop those products which only sold total of 1's on average in last 3 months?  Does that make it easier for the mod to find something?